In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pandas as pd
import numpy as np

import Models.Corn_USA_Yield as cy
import Utilities.GLOBAL as GV
import Utilities.Modeling as um
from datetime import datetime as dt

import warnings # supress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

In [8]:
scope = cy.Define_Scope()
raw_data = cy.Get_Data(scope)

In [9]:
milestones = cy.Milestone_from_Progress(raw_data)
intervals = cy.Intervals_from_Milestones(milestones)

train_DF_instr = um.Build_DF_Instructions('weighted',GV.WD_HIST, prec_units='in', temp_units='F')

In [10]:
train_df = cy.Build_Train_DF(raw_data, milestones, intervals, train_DF_instr)
model = um.Fit_Model(train_df,'Yield',GV.CUR_YEAR)
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Yield   R-squared:                       0.976
Model:                            OLS   Adj. R-squared:                  0.968
Method:                 Least Squares   F-statistic:                     122.8
Date:                Fri, 17 Jun 2022   Prob (F-statistic):           1.53e-19
Time:                        23:25:22   Log-Likelihood:                -99.986
No. Observations:                  37   AIC:                             220.0
Df Residuals:                      27   BIC:                             236.1
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                   -3

In [11]:
instructions=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F')

In [12]:
pred_df = cy.Build_Pred_DF(raw_data, milestones, instructions, dt(2022, 6, 16), GV.CUR_YEAR)
pred_df

Days to calculate: 17


,const,Trend,Yield,Planted pct on May 15th,Planting Prec,Planting Prec Squared,Jul Aug Prec,Jul Aug Prec Squared,Prec Interaction,Pollination SDD,Regular SDD
2022,1.0,2022,NaN,49.0,8.809509,77.607446,9.889243,97.797123,87.119372,32.675704,36.053016
2022,1.0,2022,NaN,49.0,8.664929,75.080994,9.889243,97.797123,85.689586,32.675704,36.053016
2022,1.0,2022,NaN,49.0,8.536073,72.864546,9.889243,97.797123,84.415300,32.675704,36.053016
2022,1.0,2022,NaN,49.0,8.390666,70.403282,9.889243,97.797123,82.977337,32.675704,36.053016
2022,1.0,2022,NaN,49.0,8.248873,68.043898,9.889243,97.797123,81.575103,32.675704,37.393963
2022,1.0,2022,NaN,49.0,8.248873,68.043898,9.764354,95.342609,80.544912,32.675704,38.178815
2022,1.0,2022,NaN,49.0,8.248873,68.043898,9.695144,93.995813,79.974005,32.675704,39.158462
2022,1.0,2022,NaN,49.0,8.248873,68.043898,9.609080,92.334413,79.264074,32.675704,41.616019
2022,1.0,2022,NaN,49.0,8.248873,68.043898,9.747110,95.006153,80.402668,32.675704,42.559861
2022,1.0,2022,NaN,49.0,8.248873,68.043898,9.626834,92.675927,79.410524,32.675704,42.754461


In [13]:
yields = model.predict(pred_df[model.params.index]).values
yields

array([175.95670447, 175.95243563, 175.93446493, 175.89815188,
       175.67883706, 175.40201457, 175.17157245, 174.72124103,
       174.82834795, 174.60938144, 174.57073973, 174.49353093,
       174.20143216, 173.98560408, 173.68789674, 173.28943479,
       172.94663851])

In [ ]:
milestones.keys()

In [ ]:
milestones['50_pct_silked']